本周只有一个代码实践题

以下代码课用来加载 预先训练好的模型,你需要只需要修改模型的存放路径即可（第二行代码）

In [1]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from torch.utils.data import DataLoader,Dataset

C:\Users\hanyi\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hanyi\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hanyi\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hanyi\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.

第一步 使用以下链接下载相应预训练模型

https://convaisharables.blob.core.windows.net/lsp/multiref/small_ft.pkl

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
weights = torch.load('Data source/small_ft.pkl')
medium_config = GPT2Config(n_embd = 768,n_layer = 12, n_head = 12)
model = GPT2LMHeadModel(medium_config)

weights['lm_head.weight'] = weights['lm_head.decoder.weight']
weights.pop('lm_head.decoder.weight',None)

model.load_state_dict(weights)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

你需要写一个推理函数，这个函数接收一个英文句子为输入，输出一个回应。

In [4]:
def generate_dialog(text):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    outputs = model(input_ids, labels=input_ids)
    loss, logits = outputs[:2]
    
    topk = torch.topk(logits[0,-1,:],1)
    next_token_id = int(topk.indices.numpy()[0])
    if next_token_id == 50256:
        print('end')
        return text
    else:
        next_word = tokenizer.decode(next_token_id)       
        text = text + next_word
        print(next_word,'NEW text: ',text,' will continue')
        return generate_dialog(text)

### 测试语句 
一轮对话   
1. Does money buy happiness ?   
2. What is the best way to buy happiness?   

一轮对话   
1. what is the meaning of a godd life ?   
2. How to be a good person ?   

In [6]:
generate_dialog("Does money buy happiness ?")

 George NEW text:  Does money buy happiness ? George  will continue
 Washington NEW text:  Does money buy happiness ? George Washington  will continue
, NEW text:  Does money buy happiness ? George Washington,  will continue
for NEW text:  Does money buy happiness ? George Washington,for  will continue
 example NEW text:  Does money buy happiness ? George Washington,for example  will continue
, NEW text:  Does money buy happiness ? George Washington,for example,  will continue
 if NEW text:  Does money buy happiness ? George Washington,for example, if  will continue
 you NEW text:  Does money buy happiness ? George Washington,for example, if you  will continue
 want NEW text:  Does money buy happiness ? George Washington,for example, if you want  will continue
 to NEW text:  Does money buy happiness ? George Washington,for example, if you want to  will continue
 know NEW text:  Does money buy happiness ? George Washington,for example, if you want to know  will continue
 what NEW text: 

"Does money buy happiness ? George Washington,for example, if you want to know what kind of happiness you're not myient.com"

In [14]:
generate_dialog("What is the best way to buy happiness?")

 advoc NEW text:  What is the best way to buy happiness? advoc  will continue
a NEW text:  What is the best way to buy happiness? advoca  will continue
es NEW text:  What is the best way to buy happiness? advocaes  will continue
ent NEW text:  What is the best way to buy happiness? advocaesent  will continue
 website NEW text:  What is the best way to buy happiness? advocaesent website  will continue
end


'What is the best way to buy happiness? advocaesent website'

In [31]:
generate_dialog("what is the meaning of a good life ? ")

 pleasure NEW text:  what is the meaning of a good life ?  pleasure  will continue
end


'what is the meaning of a good life ?  pleasure'

In [52]:
generate_dialog("How to be a good person ?")

 being NEW text:  How to be a good person ? being  will continue
 a NEW text:  How to be a good person ? being a  will continue
 good NEW text:  How to be a good person ? being a good  will continue
 person NEW text:  How to be a good person ? being a good person  will continue
end


'How to be a good person ? being a good person'